In [2]:
dropbox_url = "https://www.dropbox.com/sh/p1swf94hs2pa47g/AAAXce5SfgWKizq7rpTM7wxna/dset_allspec_150?preview=dset_allspec_150_spectrograms.nc&subfolder_nav_tracking=1&st=wvoql63r&dl=1"


In [3]:
import urllib
local_filename1 = '/content/old_data/spectzip.nc'

In [4]:
import os
os.makedirs('/content/old_data' , exist_ok = True)

In [5]:
def download_file(url, local_filename):
    try:
        # Download the file from the URL and save it locally
        urllib.request.urlretrieve(url, local_filename)
        print(f"Downloaded: {local_filename}")
    except Exception as e:
        print(f"Failed to download. Error: {e}")

# Download the file
download_file(dropbox_url, local_filename1)

Downloaded: /content/old_data/spectzip.nc


In [6]:
!unzip /content/old_data/spectzip.nc -d /content/old_data

Archive:  /content/old_data/spectzip.nc
mapname:  conversion of  failed
 extracting: /content/old_data/seis_dict.pkl  
 extracting: /content/old_data/class_dict.pkl  
 extracting: /content/old_data/dset_allspec_150_info.txt  
 extracting: /content/old_data/dset_allspec_150_chunks.csv  
 extracting: /content/old_data/dset_allspec_150_chunks_clean.nc  

Delete clean chunks file , when disk size reach maximum as we are only gonna work with spectrograms .


In [7]:
import os
os.remove('/content/old_data/spectzip.nc')

In [8]:
!pip install netCDF4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.3 MB/s eta 0:00:00


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import shutil
shutil.copy('/content/drive/MyDrive/8julyseis/spect.nc' , '/content/old_data/dset_allspec_150_spectrograms.nc')

'/content/old_data/dset_allspec_150_spectrograms.nc'

In [12]:
from netCDF4 import Dataset
import numpy as np

# Define the file path within Google Drive
file_path = '/content/old_data/dset_allspec_150_spectrograms.nc'

# Create a new NetCDF file
ncfile = Dataset(file_path, 'r')
print(ncfile)
ncfile.close()

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): side(128), traces(16831), comp(3)
    variables(dimensions): float32 mel_spectr(traces, side, side, comp), int32 class(traces), float32 distance(traces), int64 seis(traces)
    groups: class_dict


In [13]:
with open("/content/old_data/dset_allspec_150_info.txt" , 'r' ) as file :
  file_content = file.read()
  print(file_content)

Maximum distance from seismometer: 150.00 meters 
Maximum number of animals: 100 
Consider only images with single species? True
Behaviours considered: 
['Running', 'Walking', 'approach', 'charge', 'riding', 'running', 'saunter', 'waking', 'waling', 'walkig', 'walkiing', 'walkin', 'walking', 'walking.', 'walkng', 'wrestle']
Species considered: 
['ELEPHANTIDAE', 'EQUIDAE', 'BOVIDAE', 'THRESKIORNITHIDAE', 'GIRAFFIDAE', 'SUIDAE', 'HOMINIDAE', 'CAMELIDAE', 'LEPORIDAE', 'NUMIDIDAE', 'FELIDAE', 'HYAENIDAE', 'MURIDAE', 'MALACONOTIDAE', 'MUSTELIDAE', 'ESTRILDIDAE', 'CANIDAE', 'HERPESTIDAE', 'HIPPOPOTAMIDAE', 'COLUMBIDAE', 'CERCOPITHECIDAE', 'STURNIDAE', 'ORYCTEROPODIDAE', 'CISTICOLIDAE']
Class dictionary: 
{'HOMINIDAE': 1, 'ELEPHANTIDAE': 2, 'SUIDAE': 3, 'FELIDAE': 4, 'NUMIDIDAE': 5, 'BOVIDAE': 6, 'GIRAFFIDAE': 7, 'HYAENIDAE': 8, 'LEPORIDAE': 9, 'EQUIDAE': 10, 'MUSTELIDAE': 11, 'HIPPOPOTAMIDAE': 12, 'noise': 0}
Class counts: 
1     71321
6      5014
0      2978
5      1743
9       548
4       

In [14]:
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

In [15]:
InputParams = {}
InputParams["LONG_DIR"] = "/content/old_data"
InputParams["NEW_DIR"] = "/content/red_data"
InputParams["DSET_NAMES"] = ["'dset_allspec_150_spectrograms.nc'"] #["dset_allspec_150_spectrograms.nc"]
InputParams["DISTANCE"] = 50
InputParams["Datafile"] = 'dset_allspec_150_spectrograms.nc'
InputParams["DATA_PATH"] = '/content/old_data'

In [16]:
def reduce_dataset_size():
  os.makedirs(InputParams["NEW_DIR"], exist_ok=True)
  for dset in InputParams["DSET_NAMES"]:

    os.system("cp {}/*pkl {}".format(InputParams["LONG_DIR"], InputParams["NEW_DIR"]))

    print("Processing : old data")
    long_dset = nc.Dataset(os.path.join(InputParams["LONG_DIR"], InputParams["Datafile"]), "r")#add spectr file name
    inds_dist = np.where(long_dset["distance"][:]<InputParams["DISTANCE"])[0]
    new_len = len(inds_dist)

    new_dset = nc.Dataset(os.path.join(InputParams["NEW_DIR"], InputParams["Datafile"]), "w")#add spectr name
    new_dset.setncatts(long_dset.__dict__)
    for group in long_dset.groups:
            new_group = new_dset.createGroup(group)
            for kattr in long_dset.groups[group].ncattrs():
                vattr = long_dset.groups[group].getncattr(kattr)
                setattr(new_group, kattr, vattr)
    for name, dim in long_dset.dimensions.items():
            new_dset.createDimension(name, len(dim) if not name=="traces" else new_len)
    for name, var in long_dset.variables.items():
            dims = tuple([new_dset.dimensions[dim].name for dim in var.dimensions])
            x = new_dset.createVariable(name, var.datatype, dims)
            new_dset[name].setncatts(long_dset[name].__dict__)
    for i, ind in enumerate(inds_dist):
            for name, var in new_dset.variables.items():
                new_dset[name][i] = long_dset[name][ind]

            if ((i+1)%(int(new_len/10)) == 0):
                print("{} % done.".format(round(100*(i+1)/new_len)))
            if ((i+1)%(int(new_len/10)) == 0):
                print("{} % done.".format(round(100*(i+1)/new_len)))
    long_dset.close()
    new_dset.close()

In [17]:
import netCDF4 as nc
reduce_dataset_size()

Processing : old data
10 % done.
10 % done.
20 % done.
20 % done.
30 % done.
30 % done.
40 % done.
40 % done.
50 % done.
50 % done.
60 % done.
60 % done.
70 % done.
70 % done.
80 % done.
80 % done.
90 % done.
90 % done.
100 % done.
100 % done.


In [20]:
!rm -r /content/old_data


In [18]:
#checking if everything is correct
datanew  = nc.Dataset('/content/red_data/dset_allspec_150_spectrograms.nc' , 'r')
print(datanew)
var1 = datanew.variables['mel_spectr']
var2 = datanew.variables['class']
var3 = datanew.variables['seis']
var4 = datanew.variables['distance']
print(var1.shape)
datanew.close

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): side(128), traces(16831), comp(3)
    variables(dimensions): float32 mel_spectr(traces, side, side, comp), int32 class(traces), float32 distance(traces), int64 seis(traces)
    groups: class_dict
(16831, 128, 128, 3)


<bound method Dataset.close of <class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): side(128), traces(16831), comp(3)
    variables(dimensions): float32 mel_spectr(traces, side, side, comp), int32 class(traces), float32 distance(traces), int64 seis(traces)
    groups: class_dict>

In [21]:
InputParams = {}
InputParams['DATA_PATH'] = '/content/red_data'
### Dataset to use
InputParams["DBASE_NAME"] = "dset_allspec_150_spectrograms.nc"

### Size of test and validation set.
InputParams["VAL_SIZE"] = 0.1
InputParams["TEST_SIZE"] = 0.1
InputParams["stations_subset"] = ["ETA00", "NNL62", "NTA02", "STA02", "NWP05", "WTA00"]

In [22]:
def split_by_station_time():
  dbase = Dataset(os.path.join(InputParams["DATA_PATH"], InputParams["DBASE_NAME"]) , 'r')
  dbase_size = dbase.dimensions["traces"].size
  indices = np.asarray(range(dbase_size))
  path = os.path.join(InputParams["DATA_PATH"], "seis_dict.pkl")
  seis_dict = load_obj(path)
  seis_subset = [seis_dict[seis] for seis in InputParams["stations_subset"]]
  inds_seis = np.array([ind for ind in indices if dbase["seis"][ind] in seis_subset])
  sub_dbase = dbase["class"][inds_seis]
  classes = np.unique(sub_dbase)
  inds_test = []
  inds_val = []
  inds_train = []
  end_test = int(InputParams["TEST_SIZE"]*len(inds_seis))
  end_val = end_test + int(InputParams["VAL_SIZE"]*len(inds_seis))
  test = inds_seis[:end_test]
  val = inds_seis[end_test:end_val]
  train = inds_seis[end_val:]
  means = np.mean(dbase["mel_spectr"][train], axis=(0, 1, 2))
  stds = np.std(dbase["mel_spectr"][train], axis=(0, 1, 2))
  np.save(os.path.join(InputParams["DATA_PATH"], "mean_spectr_station_time.npy"), means.data)
  np.save(os.path.join(InputParams["DATA_PATH"], "std_spectr_station_time.npy"), stds.data)
  np.save(os.path.join(InputParams["DATA_PATH"], "test_station_indices.npy"), test.data)
  np.save(os.path.join(InputParams["DATA_PATH"], "val_station_indices.npy"), val.data)
  np.save(os.path.join(InputParams["DATA_PATH"], "train_station_indices.npy"), train.data)




In [23]:
import pickle

In [24]:
split_by_station_time()

In [25]:
train = np.load('/content/red_data/train_station_indices.npy')
val = np.load('/content/red_data/val_station_indices.npy')
test = np.load('/content/red_data/test_station_indices.npy')

Making DataLoader Classes

In [26]:
from torch.utils.data import Dataset

In [27]:
import torch

In [28]:
cfg = {}
cfg["DataDir"] =  '/content/red_data'
cfg["Datafile"] = 'dset_allspec_150_spectrograms.nc'

In [29]:
import torch.nn.functional as F

In [30]:
class spect_dataset(Dataset):
  def __init__(self , phase ,  InputParams = cfg ):
    """Loads dataset, indices, and normalisation.

        Parameters:
        ----------

        phase: str
            "train" or "val" phase
        """
    self.dset = nc.Dataset(os.path.join(InputParams["DataDir"] , InputParams["Datafile"]), 'r')
    self.inds = np.load(os.path.join( cfg["DataDir"]  ,"%s_station_indices.npy"%(phase)))

    self.means = np.load(os.path.join( cfg["DataDir"]  ,("mean_spectr_station_time.npy")))
    self.stds = np.load(os.path.join( cfg["DataDir"]  ,("std_spectr_station_time.npy")))
    self.phase = phase
    _, self.class_counts_all = np.unique(self.dset["class"][self.inds], return_counts=True)
    self.binary = True
    self.class_dict_all = {}
    for species in self.dset.groups["class_dict"].ncattrs():
      self.class_dict_all[self.dset.groups["class_dict"].getncattr(species)] = species
    self.class_counts = []
    self.class_dict = {}

    cls_el = self.dset.groups["class_dict"].getncattr("ELEPHANTIDAE")
    self.class_counts.append(self.class_counts_all[cls_el])
    non_el_class_counts = [count for i,count in enumerate(self.class_counts_all) if i!=cls_el]
    self.class_counts.append(np.sum(non_el_class_counts))
    self.class_dict[0] = "ELEPHANTIDAE"
    self.class_dict[1] = "OTHERS"

  def __len__(self):
    return len(self.inds)
  def returncount(self):
    return self.class_counts
  def returndict(self):
    return self.class_dict
  def __getitem__(self , index):

    ind = self.inds[index]
    cls = self.dset["class"][ind]
    if cls == 1 :
      cls = 1
    else :
      cls = 0
    spectr = (self.dset["mel_spectr"][ind] - self.means)/self.stds
    spectr = np.transpose(spectr, (2, 0, 1))
    scale_factor = 224 / 128
    x = torch.from_numpy(np.reshape(spectr[0] , (1 , 1,128,128)))
    upsampled_x = F.interpolate(x, scale_factor=scale_factor, mode='bilinear', align_corners=False)

    return torch.reshape((torch.squeeze(upsampled_x)) , (1,224,224)), torch.from_numpy(np.array([cls]))




In [31]:
dataset = spect_dataset( "train" , cfg)

In [32]:
dataset.returncount()

[5579, 2018]

In [35]:
dataset[1][0].shape

torch.Size([1, 224, 224])

In [34]:
dataset.class_counts

[5579, 2018]

In [41]:
import torch
import torch.nn as nn
import torchvision.models as models
import numpy as np

class PretrainedCNN(nn.Module):
    """CNN pretrained on Imagenet."""

    def __init__(self, num_classes, class_counts, init_bias=False, arch="resnet", pretrained=True):
        """ Add classification head to pretrained CNN."""

        super(PretrainedCNN, self).__init__()

        self.cnnlayer = nn.Conv2d(1 , 3 , kernel_size = (1,1) , stride = (1,1))

        self.model = models.squeezenet1_0(pretrained=True)
        self.model.classifier[1] = nn.Conv2d(512, 2, kernel_size=(1,1), stride=(1,1))
        self.model.num_classes = 2



    def forward(self, x):



        x = self.cnnlayer(x)
        return self.model(x)

In [42]:
def train_step(model, optimizer, criterion, inputs, labels, device, train_conf_matrix):
    model.train()
    optimizer.zero_grad()
    inputs = inputs.to(device)
    labels = labels.long().squeeze().to(device)
    preds = model(inputs)
    loss = criterion(preds, labels)
    loss.backward()
    optimizer.step()

    class_preds = torch.argmax(preds, dim=1)

    conf_batch = confusion_matrix(labels.detach().cpu().numpy(), class_preds.detach().cpu().numpy(), labels=[0,1])
    train_conf_matrix[:][:] += conf_batch
    return loss.item()

In [43]:
def val_step(model, criterion, inputs, labels, device, val_conf_matrix):
    model.eval()
    with torch.no_grad():
        inputs = inputs.to(device)
        labels = labels.long().squeeze().to(device)
        preds = model(inputs)
        loss = criterion(preds, labels)

    class_preds = torch.argmax(preds, dim=1)

    conf_batch = confusion_matrix(labels.detach().cpu().numpy(), class_preds.detach().cpu().numpy(), labels=[0,1])
    val_conf_matrix[:][:] += conf_batch

    return loss.item()


In [44]:
import sys
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
import torchvision.models as models
import torch.nn as nn
import os
import copy
from sklearn.metrics import confusion_matrix
from torch.optim import lr_scheduler

In [52]:
cfg['batch_size'] = 32
cfg['lr'] = 0.00001
cfg['l2reg'] = 0.1
cfg['scheduler'] = {}
cfg['scheduler']['step_size'] = 10
cfg['scheduler']['gamma'] = 1
cfg['epochs'] = 20
cfg['patience'] = 20


In [53]:
def setupandtrain(cfg):
  np.random.seed(42)
  torch.manual_seed(42)
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(device)
  k_datasets = {x: spect_dataset(x , cfg) for x in ['train', 'val', 'test']}
  k_dataloaders = {x: torch.utils.data.DataLoader(k_datasets[x], batch_size=cfg["batch_size"], shuffle=True) for x in ['train', 'val', 'test']}
  dataset_sizes = {x: len(k_datasets[x]) for x in ['train', 'val', 'test']}
  class_counts = k_datasets["train"].class_counts
  tot_cls = np.sum(class_counts)
  class_dict = k_datasets["train"].class_dict
  num_classes = len(class_counts)
  class_counts_val = k_datasets["val"].class_counts
  tot_cls_val = np.sum(class_counts_val)

  model = PretrainedCNN(num_classes , class_counts, init_bias=False, arch='resnet', pretrained=True)
  model.to(device)


  optimizer = optim.Adam(model.parameters(), lr = cfg['lr'], weight_decay=cfg['l2reg'])
  weights = torch.Tensor([tot_cls/class_count for class_count in class_counts]).to(device)
  total_weights = torch.sum(weights)
  weights = weights/total_weights



  criterion_train = nn.CrossEntropyLoss(weight=weights)
  criterion_val = torch.nn.CrossEntropyLoss(reduction="sum")
  criterion = torch.nn.CrossEntropyLoss()
  scheduler = lr_scheduler.StepLR(optimizer, step_size=cfg["scheduler"]["step_size"], gamma=cfg["scheduler"]["gamma"])
  print(cfg, flush=True)

  logger = Logger(cfg)

  train_loss = 0
  val_loss = 0
  species = [class_dict[i] for i in range(num_classes)]


  val_conf_matrix = pd.DataFrame(data = np.zeros((num_classes, num_classes)), columns = species)
  train_conf_matrix = pd.DataFrame(data = np.zeros((num_classes, num_classes)), columns = species)
  test_conf_matrix = pd.DataFrame(data = np.zeros((num_classes, num_classes)), columns = species)


  val_conf_matrix.index = val_conf_matrix.columns
  train_conf_matrix.index = train_conf_matrix.columns
  test_conf_matrix.index = test_conf_matrix.columns



  best_val_conf_matrix = copy.deepcopy(val_conf_matrix)
  best_model_weights = copy.deepcopy(model.state_dict())
  early_stopping = EarlyStopping(patience=cfg['patience'])


  #output untrained model metrics
  model.eval()
  for inputs, labels in k_dataloaders["train"]:
    batch_loss = val_step(model, criterion_val, inputs, labels, device, train_conf_matrix)
    train_loss += batch_loss
  for inputs, labels in k_dataloaders["val"]:
    batch_loss = val_step(model, criterion_val, inputs, labels, device, val_conf_matrix)
    val_loss += batch_loss
  train_loss = train_loss/dataset_sizes["train"]
  val_loss = val_loss/dataset_sizes["val"]
  logger.log(train_loss, val_loss, train_conf_matrix, val_conf_matrix, 0)
  print(f" Epochs 0 / {cfg['epochs']} , train_loss : {train_loss} , val_loss : {val_loss} ")
  print(val_conf_matrix, flush=True)
  print("\n", flush=True)


  #reset metrics

  val_loss = 0
  train_loss = 0
  val_conf_matrix[:][:] = np.zeros((num_classes, num_classes))
  train_conf_matrix[:][:] = np.zeros((num_classes, num_classes))
  for epoch in range(1,cfg['epochs']+1):
    model.train()
    for inputs, labels in k_dataloaders["train"]:
      batch_loss = train_step(model, optimizer, criterion_val , inputs, labels, device, train_conf_matrix)
      train_loss += batch_loss
    model.eval()
    for inputs, labels in k_dataloaders["val"]:
        batch_loss = val_step(model, criterion_val, inputs, labels, device, val_conf_matrix)
        val_loss += batch_loss
    # End of epoch callbacks
    scheduler.step()
    best_val_conf_matrix, best_model_weights = update_best_model(val_conf_matrix, best_val_conf_matrix, model, best_model_weights)
    early_stopping.early_stop(val_conf_matrix)
    train_loss = train_loss/dataset_sizes["train"]
    val_loss = val_loss/dataset_sizes["val"]
    logger.log(train_loss, val_loss, train_conf_matrix, val_conf_matrix, epoch)
    print("Epoch {}/{}, train loss = {}, valloss = {}".format(epoch, cfg['epochs'], train_loss, val_loss), flush=True)
    print(val_conf_matrix, flush=True)
    print("\n", flush=True)

    if early_stopping.stop:
      break

    val_loss = 0
    train_loss = 0
    val_conf_matrix[:][:] = np.zeros((num_classes, num_classes))
    train_conf_matrix[:][:] = np.zeros((num_classes, num_classes))
  print("Best validation confusion matrix: \n", flush=True)
  print(best_val_conf_matrix, flush=True)

  model.load_state_dict(best_model_weights)
  for inputs, labels in k_dataloaders["test"]:
    _ = val_step(model, criterion, inputs, labels, device, test_conf_matrix)
  print("Test confusion matrix from best val model: \n", flush=True)
  print(test_conf_matrix, flush=True)
  #_, _ = logger.get_all_labels_probas(model, loader_val_noshuffle, device, phase="val", save=True)
  #_, _ = logger.get_all_labels_probas(model, loader_train_noshuffle, device, phase="train", save=True)
  #_, _ = logger.get_all_labels_probas(model, loader_test_noshuffle, device, phase="test", save=True)
  return model


In [ ]:
setupandtrain(cfg)

cuda
{'DataDir': '/content/red_data', 'Datafile': 'dset_allspec_150_spectrograms.nc', 'batch_size': 32, 'lr': 1e-05, 'l2reg': 0.1, 'scheduler': {'step_size': 10, 'gamma': 1}, 'epochs': 20, 'patience': 20}


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_0_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


 Epochs 0 / 20 , train_loss : 0.5699021503990287 , val_loss : 0.51136889924993 
              ELEPHANTIDAE  OTHERS
ELEPHANTIDAE           5.0   156.0
OTHERS                 8.0   780.0


Epoch 1/20, train loss = 0.5122108514079518, valloss = 0.38018229236341505
              ELEPHANTIDAE  OTHERS
ELEPHANTIDAE          23.0   138.0
OTHERS                 8.0   780.0


Epoch 2/20, train loss = 0.41087454156497755, valloss = 0.31149578446206355
              ELEPHANTIDAE  OTHERS
ELEPHANTIDAE          43.0   118.0
OTHERS                 4.0   784.0


Epoch 3/20, train loss = 0.3552702177790132, valloss = 0.26868147242056684
              ELEPHANTIDAE  OTHERS
ELEPHANTIDAE          59.0   102.0
OTHERS                10.0   778.0


Epoch 4/20, train loss = 0.3138694104762929, valloss = 0.23148126295670568
              ELEPHANTIDAE  OTHERS
ELEPHANTIDAE          73.0    88.0
OTHERS                 5.0   783.0


Epoch 5/20, train loss = 0.2991691373972198, valloss = 0.22459234000508477
         

In [273]:
import shutil
import os

# Example path to your .nc file
nc_file_path = '/content/red_data/dset_allspec_150_spectrograms.nc'
os.makedirs('/content/drive/My Drive/8julyseis' , exist_ok= True)
# Destination folder in your Google Drive
drive_folder = '/content/drive/My Drive/8julyseis/spect.nc'

# Extracting file name
file_name = os.path.basename(nc_file_path)

# Copy the file to Google Drive
shutil.copy(nc_file_path, drive_folder)

'/content/drive/My Drive/8julyseis/spect.nc'

In [37]:
class EarlyStopping :
  #early stopping if accuracy does not improve
  def __init__(self , patience = 5):
    self.stop = False
    self.patience = max(patience , 1)
    self.counter = 0
    self.best_acc = 0
  def early_stop(self , conf_mat ):
    avg_accuracy = np.mean(np.diag(conf_mat) / np.sum(conf_mat, axis=1))
    if self.best_acc < avg_accuracy :
      self.best_acc = avg_accuracy
      self.counter = 0
    else :
      self.counter += 1
    if self.counter >= self.patience :
      self.stop = True




In [38]:
def update_best_model(conf_matrix, best_conf_matrix, model, best_model_weights):
  avg_accuracy = np.mean(np.diag(conf_matrix)/np.sum(conf_matrix, axis=1))
  if np.sum(np.sum(best_conf_matrix)) == 0:
    best_avg_accuracy = 0
  else :
    best_avg_accuracy = np.mean(np.diag(best_conf_matrix)/np.sum(best_conf_matrix, axis=1))
  if avg_accuracy>best_avg_accuracy:
        best_model_weights = copy.deepcopy(model.state_dict())
        best_conf_matrix = copy.deepcopy(conf_matrix)
  return best_conf_matrix, best_model_weights

In [39]:
def save_best_model(model, best_model_weights, cfg):
  model.load_state_dict(best_model_weights)
  torch.save(model, "best_model.pt")

In [40]:
import pandas as pd
import numpy as np
import torch

class Logger:
    def __init__(self, cfg, log_csv=True, log_tboard=True, metrics=["tp", "fp", "tn", "fn", "avg_acc", "acc", "mcc", "loss", "f1", "tpr", "fpr", "tnr", "precision"]):
        self.metrics = metrics
        self.log_csv = log_csv
        self.log_tboard = log_tboard
        if log_csv:
            self.df_log_train = pd.DataFrame(data=np.zeros((cfg['epochs']+1, len(metrics))), columns=metrics)
            self.df_log_val = pd.DataFrame(data=np.zeros((cfg['epochs']+1 , len(metrics))), columns=metrics)

    def log(self, train_loss, val_loss, train_conf_matrix, val_conf_matrix, epoch):
        for metric in self.metrics:
            if self.log_csv:
                self.df_log_train[metric][epoch] = self.compute_metric(metric, train_loss, train_conf_matrix)
                self.df_log_val[metric][epoch] = self.compute_metric(metric, val_loss, val_conf_matrix)

    def compute_metric(self, metric, loss, conf_matrix):
        """Return metric of interest."""
        tp = conf_matrix.iloc[0,0]
        fp = conf_matrix.iloc[1,0]
        tn = conf_matrix.iloc[1,1]
        fn = conf_matrix.iloc[0,1]
        tpr = tp / (tp + fn)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        precision = tp / (tp + fp)
        acc = (tp + tn) / (tp + tn + fp + fn)
        bal_acc = (tpr + tnr) / 2
        f1 = 2 * tp / (2 * tp + fn + fp)

        try:
            mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
        except RuntimeWarning:
            mcc = 0

        if metric == "tp":
            return tp
        elif metric == "fp":
            return fp
        elif metric == "tn":
            return tn
        elif metric == "fn":
            return fn
        elif metric == "tpr":
            return tpr
        elif metric == "tnr":
            return tnr
        elif metric == "fpr":
            return fpr
        elif metric == "precision":
            return precision
        elif metric == "acc":
            return acc
        elif metric == "avg_acc":
            return bal_acc
        elif metric == "f1":
            return f1
        elif metric == "mcc":
            return mcc
        elif metric == "loss":
            return loss
        else:
            raise ValueError("Metric not implemented.")

    def log_weights(self, weights):
        """write class weights to info file"""
        with open("class_weights.txt", "w") as f:
            f.write("Class weights in training set:\n")
            f.write(str(weights.detach().cpu().numpy()))

    def log_model(self, model, dataset):
        """write model summary to file."""
        torch.save(model, '/content/drive/MyDrive/SecondFinal/model.pth')

    def __del__(self):
        """Save csv file and close tensorboard summaries in destructor"""
        if self.log_csv:
            self.df_log_train.to_csv("train_metrics_log.csv")
            self.df_log_val.to_csv("val_metrics_log.csv")

    def get_all_labels_probas(self, model, dataloader, device, phase="val", save=True):
        """loop through val set and return all labels and predictions probas."""
        model.eval()
        tot_labels = []
        tot_preds = []
        for inputs, labels, _ in dataloader:
            with torch.no_grad():
                inputs = inputs.to(device)
                labels = labels.to(device)
                preds = torch.nn.functional.softmax(model(inputs), dim=-1)
                tot_labels.append(labels.detach().cpu().numpy())
                tot_preds.append(preds.detach().cpu().numpy())

        tot_labels = np.concatenate(tot_labels, axis=0)
        tot_preds = np.concatenate(tot_preds, axis=0)

        if save:
            np.save("{}_all_labels.npy".format(phase), tot_labels)
            np.save("{}_all_probas.npy".format(phase), tot_preds)

        return tot_labels, tot_preds



